In [1]:
import numpy as np
import pandas as pd

# Spam Filtering for the ENRON Spam Datasets

We will train a Naive Bayes Classifier on the ENRON Spam Datasets [1]. Naive Bayes is a popular choice for spam filtering due to its simplicity and accuracy within the spam filtering domain. It is called "naive" because we assume that word appearances are independent within an email. This is not the case but it still leads to good performance.

We represent each email as a vector of word counts, where only the 1000 most important words within the training set are considered (see "Preprocessing for the Email Classifier"). Let $F = \{t_1, t_2, \dots, t_m\}$ denote the set of words that we chose ($m = 1000$). By Bayes' theorem, the probability that an email with vector $\mathbf{x} = \langle x_1, x_2, \dots, x_m \rangle$ is *spam* is

$$
p(s \mid \mathbf{x}) = \frac{p(s) \cdot p(\mathbf{x} \mid s)}{p(s) \cdot (\mathbf{x} \mid s) + p(h) \cdot p(\mathbf{x} \mid h)},
$$

where $s$ denotes the spam category and $h$ denotes the ham category (not spam). We classify a message $\mathbf{x}$ as spam whenever $p(s \mid \mathbf{x})$ is greater than some threshold $T$. As we shall see later, varying $T$ will have different consequences.

Each email $e$ can be seen as independently picking $|e|$ words from $F$ with replacement. Therefore, $p(\mathbf{x} \mid c)$ is the multinomial distribution:

$$
p(\mathbf{x} \mid c) = p(|e|) \cdot |e|! \cdot \prod_{i=1}^m{\frac{p(t_i \mid c)^{x_i}}{x_i!}},
$$

where we have made another common assumption [1] that $|e|$ does not depend on the category $c$. This is incorrect because the length of a message can give us info about whether that message is spam. The probability of an email $\mathbf{x}$ being spam is now

$$
p(s \mid \mathbf{x}) = \frac{p(s) \cdot \prod_{i=1}^m{p(t_i \mid s)^{x_i}}}{\sum_{c \in \{s, h\}}{p(c) \cdot \prod_{i=1}^m{p(t_i \mid c)^{x_i}}}}.
$$

The empirical probability of $p(t \mid c)$ is $N_{t, c} / N_c$, where $N_{t, c}$ is the number of occurrences of word $t$ in the in the training messages of category $c,$ and $N_c$ is the total occurrences of every word within the category $c.$ We apply [laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) with $\alpha = 1$ [1] and estimate $p(t \mid c)$ as

$$
p(t \mid c) = \frac{1 + N_{t, c}}{m + N_c}.
$$

## Building the classifier

The train and test data have already been preprocessed into word count vectors based on the best 500 words information gain wise. We want to precalculate $p(s),$ $p(h),$ and all $p(t \mid c)$ in order to keep our classifier from depending on the entire training set. We build a table `p` where `p[word]['spam']` gives $p(t \mid s)$ for a given `word`. `p['ANY']['spam']` gives $p(s),$ the probability that any message is spam. `p[word]['ham']` and `p['ANY']['ham']` behave similarily.

In [2]:
train_data = pd.read_hdf('data/enron/spam_filter_train_preprocessed.h5')
test_data = pd.read_hdf('data/enron/spam_filter_test_preprocessed.h5')

In [3]:
word_counts = train_data.drop(['SPAM'], axis=1)
def probabilities(category):
    individual_occurrences = category.sum()
    all_occurrences = individual_occurrences.sum()

    return (1 + individual_occurrences) / (len(category.columns) + all_occurrences)

spam_probabilities = probabilities(word_counts[train_data['SPAM']])
ham_probabilities = probabilities(word_counts[np.logical_not(train_data['SPAM'])])

prob_spam = len(word_counts[train_data['SPAM']]) / len(word_counts)
prob_ham = 1. - prob_spam

p_spam = pd.concat([pd.DataFrame({'ANY': [prob_spam]}).transpose(), spam_probabilities])
p_ham = pd.concat([pd.DataFrame({'ANY': [prob_ham]}).transpose(), ham_probabilities])

p = pd.concat({'spam': p_spam, 'ham': p_ham}, axis=1).transpose()
p

,,ANY,http,your,more,here,no,our,all,www,com,...,reform,pr,illustrator,otcbb,assurance,opinions,macromedia,revenues,tax,value
spam,0,0.2913,0.005766,0.01161,0.003501,0.003253,0.004352,0.006224,0.005327,0.003573,0.005635,...,0.000137,0.000151,0.000183,0.000242,0.000229,0.000170,0.000314,0.000242,0.000196,0.000236
ham,0,0.7087,0.000825,0.00715,0.001197,0.001267,0.002216,0.003552,0.003618,0.000546,0.009366,...,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000019,0.000399,0.000236


We save the table for later use. 

In [4]:
p.to_hdf('resources/enron_spam_filter_table.h5', key='enron_spam_filter_table', mode='w')

## Evaluating the classifier

Let's see how the classifier would fair if the threshold for classifying spam was 0.5. This would mean that we would classify an email as spam anytime that it was more likely than being ham.

In [5]:
test_word_counts = test_data.drop(['SPAM'], axis=1)

spam_term = p.loc['spam'].drop(['ANY'], axis=1).to_numpy() ** test_word_counts
spam_term = p['ANY']['spam'].to_numpy() * spam_term.prod(axis=1)

ham_term = p.loc['ham'].drop(['ANY'], axis=1).to_numpy() ** test_word_counts
ham_term = p['ANY']['ham'].to_numpy() * ham_term.prod(axis=1)

prob_spam = spam_term / (spam_term + ham_term)
prob_spam = prob_spam.fillna(p['ANY']['spam'])

is_spam = prob_spam > 0.5

correct = test_data['SPAM'] == is_spam

accuracy = correct.sum() / len(test_data) * 100

print('Classification Accuracy: ' + f'{accuracy:.1f}' + '%')

Classification Accuracy: 88.6%


We get a classification accuracy of 88.6% for $T = 0.5$. This may be decent enough, but lets look at the messages that were classified incorrectly.

In [6]:
incorrect = test_data.loc[test_data['SPAM'] != is_spam]

filtered_ham = incorrect['SPAM'].sum() / np.logical_not(test_data['SPAM']).sum() * 100
unfiltered_spam = np.logical_not(incorrect['SPAM']).sum() / test_data['SPAM'].sum() * 100

print('Percentage of ham incorrectly filtered: ' + f'{filtered_ham:.1f}' + '%. Percentage of spam unfiltered: ' + f'{unfiltered_spam:.1f}' + '%.')

Percentage of ham incorrectly filtered: 13.4%. Percentage of spam unfiltered: 6.2%.


As we can see, most of the error in our filter comes from incorrectly filtering out legitimate messages. Our filter did a good job at catching spam, only letting 6.2% get through, but this comes at the cost of filtering out a large amount of legitimate messages. An email service would not be very user friendly if the user's messages had more than a 1 in 10 chance of being put into the spam folder. Therefore, we need a better measure of the filter's performance.

## Precision vs. Recall

We want our filter to correctly classify spam messages as being spam, but we also want the filter to not incorrectly classify ham messages as spam. This requirement can be measured with the precision and recall metrics. Precision is the ratio of true positives (spam classified as spam) over all predicted positives (all messages classified as spam). Precision will be high when there are few false positives (ham classified as spam). Recall is ratio of true positives over all actual positives (all actual spam). Recall will be high when there are few false negatives (spam classified as ham). For this problem, we care more about precision then we do about recall. It is ok to let more spam get through the filter if it means legitimate messages will have a small chance of being flagged as spam.

We evaluate the filter's precision and recall for $T = 0.5, 0.75, 0.95, 0.96, 0.97, 0.98$ and $0.99$. We will make a tradeoff between false positives and false negatives. We can't *only* care about precision, because an email service that is flooded with spam can be just as unfriendly as one where legitimate messages are flagged as spam.

In [7]:
def precision_and_recall(actual_spam, predicted_prob_spam, threshold):
    predicted_spam = predicted_prob_spam > threshold
    
    precision = actual_spam[predicted_spam].sum() / predicted_spam.sum()
    recall = predicted_spam[actual_spam].sum() / actual_spam.sum()
    
    return precision, recall

pr_5 = precision_and_recall(test_data['SPAM'], prob_spam, 0.5)
pr_75 = precision_and_recall(test_data['SPAM'], prob_spam, 0.75)
pr_95 = precision_and_recall(test_data['SPAM'], prob_spam, 0.95)
pr_96 = precision_and_recall(test_data['SPAM'], prob_spam, 0.96)
pr_97 = precision_and_recall(test_data['SPAM'], prob_spam, 0.97)
pr_98 = precision_and_recall(test_data['SPAM'], prob_spam, 0.98)
pr_99 = precision_and_recall(test_data['SPAM'], prob_spam, 0.99)

pr = pd.DataFrame({'T = 0.5': pr_5, 'T = 0.75': pr_75, 'T = 0.95': pr_95, 'T = 0.96': pr_96, 'T = 0.97': pr_97, 'T = 0.98': pr_98, 'T = 0.99': pr_99}, 
                  index=['precision', 'recall'])
pr

,T = 0.5,T = 0.75,T = 0.95,T = 0.96,T = 0.97,T = 0.98,T = 0.99
precision,0.912621,0.919192,0.939759,0.939024,0.950000,0.948718,0.972222
recall,0.652778,0.631944,0.541667,0.534722,0.527778,0.513889,0.486111


We see the most improvement in precision from 0.95-0.99. A 95% precision should be adequate, so we choose $T = 0.97$. This leaves almost 50% of spam unfiltered, and the precision could definitely be better. In a real world scenario, the best thing to do here would probably be to collect better data and try different techniques to build a better filter, rather than making more tradeoffs.

## References

[1] Vangelis Metsis, Ion Androutsopoulos, and Georgios Paliouras. Spam Filtering with Naive Bayes - Which Naive Bayes? *CEAS 2006 - Third Conference on Email and Anti-Spam*, July 27-28, 2006, Mountain View, California USA